## [Commands](https://ardupilot.org/dev/docs/copter-commands-in-guided-mode.html)

In [1]:
from pymavlink import mavutil
import subprocess
import time
import os

# Custom Modules
from mission_flow import Step,Action

Kill related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Launch Copter (ardupilot)

In [3]:
sim_vehicle_path = os.path.expanduser("~/ardupilot/Tools/autotest/sim_vehicle.py")
vehicle_cmd = f"python3 {sim_vehicle_path} -v ArduCopter" 
# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--', 'bash', '-c', 'pytho...>

Conect to the device

In [4]:
conn = mavutil.mavlink_connection('udp:127.0.0.1:14550')

conn.wait_heartbeat()
print("Heartbeat received. Connected to the vehicle.")

Heartbeat received. Connected to the vehicle.


In [5]:
EKF_FLAGS = {
    "ATTITUDE": mavutil.mavlink.EKF_ATTITUDE,
    "VELOCITY_HORIZ": mavutil.mavlink.EKF_VELOCITY_HORIZ,
    "POS_HORIZ_ABS": mavutil.mavlink.EKF_POS_HORIZ_ABS,
    "POS_VERT_ABS": mavutil.mavlink.EKF_POS_VERT_ABS,
}

REQUIRED_SENSORS = {
    "3D_GYRO": mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_GYRO,
    "3D_ACCEL": mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_ACCEL,
    "3D_MAG": mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_MAG,
    "ABS_PRESSURE": mavutil.mavlink.MAV_SYS_STATUS_SENSOR_ABSOLUTE_PRESSURE,
    "GPS": mavutil.mavlink.MAV_SYS_STATUS_SENSOR_GPS,
}

class StepFailed(Exception):
    """Custom exception for step-level failures."""
    pass



def check_noarmed(blocking=False):
    """Check if UAV is disarmed (not already armed)."""
    msg = conn.recv_match(type="HEARTBEAT", blocking=blocking)
    if not msg:
        return False  # Normal: still waiting for heartbeat

    is_armed = msg.base_mode & mavutil.mavlink.MAV_MODE_FLAG_SAFETY_ARMED
    if is_armed:
        raise StepFailed("UAV is already armed")
    
    return True

def check_ekf_status(blocking=False):
    """Check if EKF is fully ready (attitude, velocity, position)."""
    msg = conn.recv_match(type="EKF_STATUS_REPORT", blocking=blocking)
    if not msg:
        return False  # Normal: still waiting for EKF report

    flags = msg.flags
    missing = [name for name, bit in EKF_FLAGS.items() if not flags & bit]

    if missing:
        print(f"EKF is not ready — missing: {', '.join(missing)}")

    return True

def check_gps_status(blocking=False):
    """Check if GPS fix is good enough (3D fix or better)."""
    msg = conn.recv_match(type="GPS_RAW_INT", blocking=blocking)
    if not msg:
        return False  # Normal: GPS still acquiring

    if msg.fix_type < 3:
        raise StepFailed(f"GPS fix too weak (fix_type = {msg.fix_type})")

    return True

def check_sys_status(blocking=False):
    """Check system status: battery and sensors."""
    msg = conn.recv_match(type="SYS_STATUS", blocking=blocking)
    if not msg:
        return False  # Normal: no system status yet

    if msg.battery_remaining < 20:
        raise StepFailed(f"Battery too low ({msg.battery_remaining}%)")

    sensors = msg.onboard_control_sensors_health
    missing = [name for name, bit in REQUIRED_SENSORS.items() if not sensors & bit]

    if missing:
        raise StepFailed(f"Missing or unhealthy sensors: {', '.join(missing)}")

    return True





check_noarmed_step = Step(
    name="Check no armed",   
    check_fn=check_noarmed   # Your function goes here
)

check_ekf_status_step = Step(
    name="check EKF status",   
    check_fn=check_ekf_status   # Your function goes here
)

check_gps_status_step = Step(
    name="check GPS status",   
    check_fn=check_gps_status   # Your function goes here
)

check_sys_status_step = Step(
    name="check system status",   
    check_fn=check_sys_status  # Your function goes here
)

In [6]:
# while check_noarmed_step.state in ["NOT_STARTED","IN_PROGRESS"]:
#     check_noarmed_step.run()

# while check_gps_status_step.state in ["NOT_STARTED","IN_PROGRESS"]:
#     check_gps_status_step .run()
# while check_sys_status_step.state in ["NOT_STARTED","IN_PROGRESS"]:
#     check_sys_status_step .run()
# while check_ekf_status_step.state in ["NOT_STARTED","IN_PROGRESS"]:
#     check_ekf_status_step.run()

In [7]:
pre_arm=Action(name='pre arm')
pre_arm.add_step(check_noarmed_step)
pre_arm.add_step(check_gps_status_step)
pre_arm.add_step(check_sys_status_step)
pre_arm.add_step(check_ekf_status_step)

In [8]:
while pre_arm.state in ["NOT_STARTED","IN_PROGRESS"]:
    pre_arm.run()

▶️ Starting Action: pre arm
▶️ Starting Step: Check no armed
✅ Step: Check no armed is done
▶️ Starting Step: check GPS status
✅ Step: check GPS status is done
▶️ Starting Step: check system status
✅ Step: check system status is done
▶️ Starting Step: check EKF status
EKF is not ready — missing: POS_HORIZ_ABS
✅ Step: check EKF status is done
✅ Action: pre arm is done


## Prearmed Comands

## Movement Commands

These commands can be used to control the vehicle’s position, velocity or attitude while in [Guided Mode](https://ardupilot.org/copter/docs/ac2_guidedmode.html#ac2-guidedmode)

In [9]:
mavutil.mavlink.EKF_POS_HORIZ_ABS

16